<a href="https://colab.research.google.com/github/selvavivaan/Machine-Learning-selva/blob/main/Generative_AI_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests
from flask import Flask, request, jsonify

app = Flask(__name__)

def extract_text_from_webpage(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        texts = soup.find_all(text=True)
        visible_texts = filter(lambda x: x.strip(), texts)
        webpage_text = " ".join(visible_texts)
        return webpage_text
    except Exception as e:
        print(f"Error fetching webpage content: {e}")
        return None

def answer_question_on_webpage(webpage_text, question):
    qa_pipeline = pipeline("question-answering")

    answer = qa_pipeline(question=question, context=webpage_text)

    return answer['answer'] if answer['score'] > 0.5 else "I don't know the answer"

@app.route('/answer_question', methods=['POST'])
def answer_question():
    request_data = request.get_json()

    if 'url' not in request_data or 'question' not in request_data:
        return jsonify({"error": "Invalid input format. 'url' and 'question' are required."}), 400

    url = request_data['url']
    question = request_data['question']

    webpage_text = extract_text_from_webpage(url)

    if webpage_text is None:
        return jsonify({"error": "Failed to fetch webpage content."}), 500

    answer = answer_question_on_webpage(webpage_text, question)

    return jsonify({"answer": answer})

#Run the Flask web application
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [12]:
from transformers import pipeline
from flask import Flask, request, jsonify

app = Flask(__name__)

def generate_marketing_content(topic, format_type):
    text_generator = pipeline("text-generation", model="EleutherAI/gpt-neo-2.7B")

    format_prompts = {
        "linkedin post": f"Excited to share insights on {topic}! 🚀 With its innovative capabilities, {topic} is revolutionizing various industries, from creative arts to healthcare. Its potential to generate realistic images, text, and even music is reshaping the way we create and interact with technology. Let's explore the endless possibilities and opportunities this cutting-edge technology offers. #{topic.replace(' ', '')} #Innovation #TechRevolution 🤖💡"
    }

    if format_type in format_prompts:
        generated_text = text_generator(format_prompts[format_type], max_length=150, num_return_sequences=1)[0]['generated_text']
        return generated_text
    else:
        return "Invalid format type. Supported formats: linkedin post"

@app.route('/generate_content', methods=['POST'])
def generate_content():
    request_data = request.get_json()

    if 'topic' not in request_data or 'format' not in request_data:
        return jsonify({"error": "Invalid input format. 'topic' and 'format' are required."}), 400

    topic = request_data['topic']
    format_type = request_data['format']

    generated_text = generate_marketing_content(topic, format_type)

    return jsonify({"text": generated_text})

#Run the Flask web application
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
